In [47]:
%pylab inline
%matplotlib inline
import regain
reload(regain)
from regain.datasets import is_pos_def, is_pos_semidef
from regain.datasets import generate_ma_xue_zou
from regain.plot import plot_graph_with_latent_variables
from sklearn.datasets import make_sparse_spd_matrix
import networkx as nx
import scipy.sparse

Populating the interactive namespace from numpy and matplotlib


In [49]:
n_dim_latent = 2
n_dim_observed = 10

In [327]:
A = np.eye(5)
from regain.datasets import is_pos_def

is_pos_def(A)

True

In [44]:
K_HO = np.zeros((n_dim_latent, n_dim_observed))
for i in range(n_dim_latent):
    percentage = int(n_dim_observed * 0.8)
    indices = np.random.randint(0, high=n_dim_observed, size=percentage)
    K_HO[i, indices] = np.random.rand(percentage)#*0.12

In [328]:
L = K_HO.T.dot(K_HO)
np.linalg.matrix_rank(L)
#u, s, v = np.linalg.svd(L)
#s_new = np.maximum(s, np.zeros(s.shape))
#L_new = u.dot(np.diag(s_new)).dot(v.T)

2

In [329]:
is_pos_semidef(L_new)

False

In [330]:
L

array([[ 0.77956858,  0.76103752,  0.        ,  0.        ,  0.        ,
         0.7692454 ,  0.        ,  0.78063216,  0.        ,  0.1435951 ],
       [ 0.76103752,  0.87232496,  0.        ,  0.3083947 ,  0.26275728,
         0.75095973,  0.11415563,  0.76207582,  0.10937663,  0.49592646],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.3083947 ,  0.        ,  0.73511173,  0.6263271 ,
         0.        ,  0.27210955,  0.        ,  0.26071798,  0.84797872],
       [ 0.        ,  0.26275728,  0.        ,  0.6263271 ,  0.53364083,
         0.        ,  0.23184174,  0.        ,  0.22213594,  0.7224916 ],
       [ 0.7692454 ,  0.75095973,  0.        ,  0.        ,  0.        ,
         0.75905892,  0.        ,  0.7702949 ,  0.        ,  0.14169358],
       [ 0.        ,  0.11415563,  0.        ,  0.27210955,  0.23184174,
         0.        ,  0.10072429,  0.        

In [350]:
theta = np.diag(np.ones(n_dim_observed))
for i in range(n_dim_observed-1):
    l = list(np.arange(i+1,n_dim_observed))
    indices = np.random.choice(l, 2)
    theta[i, indices] = 0.25
    theta[indices, i] = 0.25
    

In [351]:
theta += np.diag(np.sum(L, axis=1))

<module 'regain.prox' from '/home/fede/src/slipguru/regain/regain/prox.pyc'>

In [352]:
print theta

[[ 4.23407875  0.          0.25        0.          0.          0.          0.
   0.          0.25        0.        ]
 [ 0.          5.43700872  0.          0.25        0.          0.          0.
   0.          0.          0.25      ]
 [ 0.25        0.          1.          0.          0.25        0.          0.
   0.          0.          0.25      ]
 [ 0.          0.25        0.          4.05063978  0.          0.          0.
   0.25        0.          0.25      ]
 [ 0.          0.          0.25        0.          3.59919448  0.          0.
   0.          0.25        0.25      ]
 [ 0.          0.          0.          0.          0.          4.19125252
   0.          0.          0.25        0.        ]
 [ 0.          0.          0.          0.          0.          0.
   2.12922728  0.25        0.25        0.        ]
 [ 0.          0.          0.          0.25        0.          0.          0.25
   4.23849109  0.25        0.25      ]
 [ 0.25        0.          0.          0.          0.2

In [353]:
theta_observed = theta -  L #? che si fa
theta_observed

array([[ 3.45451017, -0.76103752,  0.25      ,  0.        ,  0.        ,
        -0.7692454 ,  0.        , -0.78063216,  0.25      , -0.1435951 ],
       [-0.76103752,  4.56468376,  0.        , -0.0583947 , -0.26275728,
        -0.75095973, -0.11415563, -0.76207582, -0.10937663, -0.24592646],
       [ 0.25      ,  0.        ,  1.        ,  0.        ,  0.25      ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.25      ],
       [ 0.        , -0.0583947 ,  0.        ,  3.31552805, -0.6263271 ,
         0.        , -0.27210955,  0.25      , -0.26071798, -0.59797872],
       [ 0.        , -0.26275728,  0.25      , -0.6263271 ,  3.06555365,
         0.        , -0.23184174,  0.        ,  0.02786406, -0.4724916 ],
       [-0.7692454 , -0.75095973,  0.        ,  0.        ,  0.        ,
         3.43219361,  0.        , -0.7702949 ,  0.25      , -0.14169358],
       [ 0.        , -0.11415563,  0.        , -0.27210955, -0.23184174,
         0.        ,  2.02850299,  0.25      

In [354]:
is_pos_def(theta_observed)

True

In [355]:
is_pos_def(theta)

True

In [356]:
d = np.diag(theta_observed).reshape(1, theta_observed.shape[0])
d = 1. / np.sqrt(d)

theta_observed *= d
theta_observed *= d.T

In [358]:
np.linalg.inv(theta_observed)

array([[  1.39350751e+00,   4.80672025e-01,  -2.12680786e-01,
         -2.19806023e-02,   7.76393328e-02,   5.50976163e-01,
          4.64497213e-04,   5.49700903e-01,  -2.21663429e-01,
          1.14809553e-01],
       [  4.80672025e-01,   1.28425693e+00,  -1.00081154e-01,
          2.87146054e-02,   1.32755955e-01,   4.66780295e-01,
          4.14708561e-02,   4.57838265e-01,  -8.69104478e-02,
          1.33572867e-01],
       [ -2.12680786e-01,  -1.00081154e-01,   1.08151420e+00,
         -6.91627732e-02,  -2.05121918e-01,  -9.34220461e-02,
         -4.58855315e-02,  -7.59922407e-02,   2.82317687e-02,
         -1.91530986e-01],
       [ -2.19806023e-02,   2.87146054e-02,  -6.91627732e-02,
          1.13787760e+00,   2.84544081e-01,  -2.95076523e-02,
          1.77277345e-01,  -1.25571200e-01,   1.21218594e-01,
          2.55367377e-01],
       [  7.76393328e-02,   1.32755955e-01,  -2.05121918e-01,
          2.84544081e-01,   1.14395990e+00,   5.47780877e-02,
          1.67840092e-01